In [1]:
class Ville:
    output_dir     = "./Villes/"
    nb_d_instance  = 0
    timestamp      = 0
            
    def __init__(self, nb_de_cyclistes, Prestataire):
        from Cycliste import Cycliste
        from Position import Position
        from Cycliste import Casseur
        from Logger import Logger
        from datetime import datetime
        import os
        Ville.nb_d_instance += 1
        
        self.id          = Ville.nb_d_instance
        self.prestataire = Prestataire
        self.x_max       = Prestataire.largeur -2
        self.y_max       = Prestataire.hauteur-2
        self.logger      = Logger(os.path.join(Ville.output_dir, "ville_{id}.csv".format(id=self.id)))
        self.cycliste_details = [ 'id', 'vitesse_a_pied', 'vitesse_a_velo', 'home', 
                                 "travail", "sportif", "casseur", "statut", "salaire",
                                "sexe", "age", "sportivite", "velo_perf_minimale"]
        self.logger.log(",".join(self.cycliste_details))
        
        self.statuts     = {"cadre"                 : (30000, 120000), 
                            "employe"               : (20000, 60000), 
                            "technicien_de_surface" : (18000, 30000) , 
                             "éboueur"              : (18000, 25000) ,  
                             "professeur"           : (23000, 35000) ,  
                             "reserviste"           : (30000, 35000) ,  }
        self.jour_ouvres = {
                            "cadre"                 : [0,1,2,3,4    ], 
                            "employe"               : [0,1,2,3,4    ], 
                            "technicien_de_surface" : [          5,6], 
                            "éboueur"               : [0,  2,  4,  6], 
                            "professeur"            : [0,1  ,3,4    ], 
                            "reserviste"            : [0,1  ,3,4,5,6], 
                             }
        self.cyclistes   = [Cycliste(Position.get_random_position(self.x_max, self.y_max), 
                                     Position.get_random_position(self.x_max, self.y_max), 
                                     self)
                                for i in range(nb_de_cyclistes)]
        self.cyclistes.extend( [Casseur(Position.get_random_position(self.x_max, self.y_max), 
                                        Position.get_random_position(self.x_max, self.y_max), 
                                        self)
                                   for i in range(round(nb_de_cyclistes/20))])
        self.ax = None
        self.timestamp = datetime.strptime("01/01/2018 00:00:00", "%d/%m/%Y %H:%M:%S")
        self.log()
        
    def ajoute_cycliste(self, nb_de_cyclistes):
        from Cycliste import Cycliste
        from Position import Position
        from Cycliste import Casseur
        import random
        
        self.cyclistes.extend( [Cycliste(Position.get_random_position(self.x_max, self.y_max), 
                                         Position.get_random_position(self.x_max, self.y_max), 
                                         self)
                                   for i in range(nb_de_cyclistes + random.randint(0, 10))])
        
        self.cyclistes.extend( [Casseur(Position.get_random_position(self.x_max, self.y_max), 
                                        Position.get_random_position(self.x_max, self.y_max), 
                                        self)
                                   for i in range(round(nb_de_cyclistes/20))])
        
    def __str__(self):
        return "\n".join([str(self.prestataire),
                   "nb de cycliste = ", str(len(self.cyclistes))])
    def log(self):
        for c in self.cyclistes:
            data = [str(c.__dict__[detail])  for detail in self.cycliste_details  ]
            message = ",".join(data)
            self.logger.log(message)
            
    def get_salaire(self, position, statut, sexe ):
        from Position import Position
        import numpy as np
        maxi    = self.statuts[statut][1] 
        mini    = self.statuts[statut][0]
        milieu   = Position(self.x_max/2, self.y_max/2)
        distance = Position.get_distance(milieu, position)
        salaire  = max(maxi / (1 + distance/3) , mini)
        if sexe == "H":
            salaire = np.random.normal(1.2, 0.3) * salaire
        if sexe == "F":
            salaire = np.random.normal(1, 0.2) * salaire
            
        return salaire 
        
    def reveil(self, nb_de_minutes=1):
        from datetime import timedelta
        self.timestamp = self.timestamp + timedelta(minutes=nb_de_minutes)
        list(map(lambda cycliste   :   cycliste.avancer(self.timestamp), self.cyclistes               ))
        list(map(lambda reparateur : reparateur.avancer(self.timestamp), self.prestataire.reparateurs ))
            
    def faire_avancer(cycliste):
        cycliste.avancer(Ville.timestamp)
        
    def reveil_multiproc(self, n_jobs=1):
        import multiprocessing
        self.timestamp += 1
        Ville.timestamp = self.timestamp
        pool            = multiprocessing.Pool(n_jobs)
        pool.map(Ville.faire_avancer, self.cyclistes)
        

        
    def get_cycliste_workplaces(self):
        import pandas as pd
        infos = { "longitudes"  : [cycliste.travail.longitude for cycliste in self.cyclistes],
                  "lattitude"   : [cycliste.travail.lattitude for cycliste in self.cyclistes],
                  "id_cycliste" : [cycliste.id                for cycliste in self.cyclistes]}
        return pd.DataFrame(infos)

    def get_cycliste_homes(self):
        import pandas as pd
        infos = { "longitudes" : [cycliste.home.longitude for cycliste in self.cyclistes],
                  "lattitude"  : [cycliste.home.lattitude for cycliste in self.cyclistes],
                  "id_cycliste" : [cycliste.id            for cycliste in self.cyclistes]}
        return pd.DataFrame(infos)

        
    def get_cycliste_positions(self):
        import pandas as pd
        infos = { "longitudes" : [x.position.longitude for x in self.cyclistes],
                  "lattitude"  : [x.position.lattitude for x in self.cyclistes],
                  "id_cycliste": [x.id                 for x in self.cyclistes]}
        return pd.DataFrame(infos)

    def get_topology(self):
        import pandas as pd
        infos = { "longitudes" : [x.position.longitude for x in self.prestataire.reseau],
                  "lattitude"  : [x.position.lattitude for x in self.prestataire.reseau]}
        return pd.DataFrame(infos)
    def get_drawing_data(self):
        cyclistes  = self.get_cycliste_positions()
        stations   = self.get_topology()
        homes      = self.get_cycliste_homes()
        workplaces = self.get_cycliste_workplaces()
        return cyclistes, stations, homes, workplaces

    def draw_map(self,ax=None):
        import matplotlib.pyplot as plt
        cyclistes, stations, homes, workplaces = self.get_drawing_data()
        if not ax:
            ax = cyclistes.plot.scatter(x="longitudes", y="lattitude"      , c=cyclistes["id_cycliste"]  , 
                                        marker="1"    , colormap='viridis' , colorbar=True      )

            stations.plot.scatter(   x="longitudes", y="lattitude", c="grey"   , marker="D", ax=ax , alpha=0.1)
            homes.plot.scatter(      x="longitudes", y="lattitude", c="green"  , marker="s", ax=ax )
            workplaces.plot.scatter( x="longitudes", y="lattitude", c="red"    , marker="x", ax=ax )
        else:
            cyclistes.plot.scatter(  x="longitudes", y="lattitude", 
                                   c=cyclistes["id_cycliste"], 
                                   marker="1" , colormap='viridis', ax=ax,  colorbar=False             )

        return ax
    def get_historique_positions_cycliste(self, jitter=False):
        import pandas as pd
        return pd.concat([x.get_historique(jitter=jitter) for x in self.cyclistes])
    
    def get_personal_data_cyclistes(self):
        import pandas as pd
        return pd.concat([x.get_personal_data() for x in self.cyclistes])
    
        